In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils import show_recipe

C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("mbien/recipenlg")
model = AutoModelForCausalLM.from_pretrained("mbien/recipenlg")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Prompt for recipe generation
prompt = (
    "Give me a healthy recipe using the following ingredients, you don't have to use all of them:"
    "<RECIPE_START>"
    "<INPUT_START>milk<NEXT_INPUT>cream<NEXT_INPUT>strawberry<NEXT_INPUT>melon<NEXT_INPUT>honey<NEXT_INPUT>salmon<INPUT_END>"
)

# Tokenize the input
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    return_attention_mask=True,
)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

# Generate output
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=512,
    num_beams=5,
    no_repeat_ngram_size=2,  # optional: avoid repeating bigrams
    early_stopping=True,
    eos_token_id=tokenizer.convert_tokens_to_ids("<RECIPE_END>"),
)

# Decode and print the generated recipe
raw_recipe = tokenizer.decode(outputs[0])
show_recipe(raw_recipe)

Setting `pad_token_id` to `eos_token_id`:50266 for open-end generation.


## Recipe Title: Raspberry-Melon Chutney

Inputs (base ingredients):
   - milk
   - cream
   - strawberry
   - melon
   - honey
   - salmon

Ingredients:
   - 1 cup whole milk
   - 1/2 cup half-and-half cream
   - 3/4 cup fresh or frozen unsweetened raspberries, thawed
   - 2/3 cup sliced strawberries, hulled and halved
   - 4 cups cubed peeled honeydew melon (about 2 pounds)
   - 8 ounces sliced salmon, cut into 1-inch pieces

Instructions:
   1. Combine first 4 ingredients in a large saucepan; bring to a boil. Reduce heat, and simmer, uncovered, 10 minutes or until slightly thickened, stirring occasionally. Remove from heat; stir in fruit. Cool to room temperature. Cover and chill at least 2 hours.
